In [25]:
import numpy as np
import numpy.linalg as la
from cmath import sqrt
from matplotlib import pyplot as plt
import pandas as pd
from scipy import linalg as sla

A = np.array([[1,3,0,0],[0,1,3,0],[0,0,1,3],[0,0,0,0]])
AD = np.array([[1,-3,9,81],[0,1,-3,-18],[0,0,1,3],[0,0,0,0]])
B = np.array([[1,1,3],[5,2,6],[-2,-1,-3]])
BD = np.zeros((3,3))

Question #1

In [21]:
def prob1(A,AD,k):
    if np.allclose(A@AD,AD@A) == True and np.allclose(la.matrix_power(A,k+1)@AD,la.matrix_power(A,k)) == True and np.allclose(AD@A@AD,AD) == True:
        return True
    return False

print(np.allclose(A@AD,AD@A))
prob1(A,AD,3)


True


True

Question #2

In [26]:
def prob2(A,tol):
    (n,n) = A.shape
    f = lambda x: abs(x) > tol
    g = lambda x: abs(x) <= tol
    Q1,S,k1 = sla.schur(A, sort=f)
    Q2,T,k2 = sla.schur(A, sort=g)
    U = np.column_stack((S[:,:k1], T[:,:(n-k1)]))
    Uinv = la.inv(U)
    V = Uinv@A@U
    Z = np.zeros((n,n))
    if k1 != 0:
        M = la.inv(V[:k1,:k1])
        Z[:k1,:k1] = M
    return U@Z@Uinv

prob2(A,0.0001)

array([[  1.,  -3.,   9.,  81.],
       [  0.,   1.,  -3., -18.],
       [  0.,   0.,   1.,   3.],
       [  0.,   0.,   0.,   0.]])

Question #3

In [34]:
def prob3(A):
    n,n = A.shape
    D = np.diag(np.sum(A, axis=0))
    L = D - A
    I = np.eye(n)
    R = np.zeros((n,n))
    for i in range(n):
        B = L.copy()
        B[:,i] = I[:,i]
        Drazin = prob2(B,0.001)
        R[:,i] = np.diagonal(Drazin)
        R[i,i] = 0
    return R

G = np.array([[0,1],[1,0]])
prob3(G)

array([[ 0.,  1.],
       [ 1.,  0.]])

Question #4

In [57]:
class LinkPredictor(object):
    
    # Updated with improved functions from wk4 PageRank
    
    def __init__(self,file):
        self.names = np.loadtxt(file, dtype=str,delimiter=",")
        self.network = self.mapper(self.names)
        self.adj = self.adjfunc(self.network)
        self.rest = self.resistance(self.adj)
    
    def mapper(self,A):
        B = -1*np.ones(A.shape)
        c1 = 0
        for i in range(0,len(A)):
            B[i,0] = c1
            k = 0
            for j in range(0,i):
                if A[i,0] == A[j,0]:
                    B[i,0] = B[j,0]
                    k = k + 1
            if k == 0:
                c1 = c1 + 1
        c2 = 0
        for i in range(0,len(A)):
            k = 0
            for j in range(0,len(A)):
                if A[i,1] == A[j,0]:
                    B[i,1] = B[j,0]
                    k = k + 1
            if k == 0:
                for b in range(0,i):
                    if A[i,1] == A[b,1]:
                        B[i,1] = B[b,1]
                        k = k + 1
            if k == 0:
                B[i,1] = c1
                c1 = c1 + 1
        C = B.astype(int)
        return C

    def adjfunc(self,mat):
        N = len(set(list(mat[:,0]) + list(mat[:,1])))
        adj = np.zeros((N,N))
        for i,node in enumerate(mat):
            adj[node[0],node[1]] = 1
        return adj
    
    def resistance(self,A):
        n,n = A.shape
        D = np.diag(np.sum(A, axis=0))
        L = D - A
        I = np.eye(n)
        R = np.zeros((n,n))
        for i in range(n):
            B = L.copy()
            B[:,i] = I[:,i]
            Drazin = self.Drazin(B)
            R[:,i] = np.diagonal(Drazin)
            R[i,i] = 0
        return R

    def Drazin(self,A,tol=0.0001):
        (n,n) = A.shape
        f = lambda x: abs(x) > tol
        g = lambda x: abs(x) <= tol
        Q1,S,k1 = sla.schur(A, sort=f)
        Q2,T,k2 = sla.schur(A, sort=g)
        U = np.column_stack((S[:,:k1], T[:,:(n-k1)]))
        Uinv = la.inv(U)
        V = Uinv@A@U
        Z = np.zeros((n,n))
        if k1 != 0:
            M = la.inv(V[:k1,:k1])
            Z[:k1,:k1] = M
        return U@Z@Uinv

        
Link = LinkPredictor("social_network.csv")
print(Link.rest)

[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 1.          0.          1.         ...,  1.          1.          1.        ]
 [ 0.5         0.5         0.         ...,  0.5         0.5         0.5       ]
 ..., 
 [ 0.5         0.5         0.5        ...,  0.          0.5         0.5       ]
 [ 0.5         0.5         0.5        ...,  0.5         0.          0.5       ]
 [ 0.05882353  0.05882353  0.05882353 ...,  0.05882353  0.05882353  0.        ]]


Question #5